In [25]:
import pandas as pd
import datetime

class SOT(object):
    def __init__(self, sot_file):
        self.sot_df = pd.read_csv(sot_file)
        self.sot_df['Date'] = pd.to_datetime(self.sot_df['Date'], dayfirst=True)
        self.sot_df['budget_date'] = pd.to_datetime(self.sot_df['budget_date'], dayfirst=True)
        
    def __call__(self):
        return self.sot_df

    def save_csv(self):
        file_name = 'SOT - {}.csv'.format(datetime.datetime.now().strftime("%y.%m.%d - %H.%M")) 
        self.sot_df.to_csv(file_name, index=False)
        print(file_name)
    
    def update_budget(self, budget_records):
        return budget_records

    def prep_new_bank(self, csv_location):
        new_bank = pd.read_csv(csv_location)
        new_bank['Date'] = pd.to_datetime(new_bank['Date'], dayfirst=True)
        new_bank.columns = [col.strip() for col in new_bank]
        return new_bank.sort_index()
    
    def update_bank(self, csv_location):
        new_bank_df = self.prep_new_bank(csv_location)
        updated_sot = self.sot_df
        for n, row in new_bank_df.iterrows():
            updated_sot = self.update_sot(row, new_bank_df, updated_sot)
        self.sot_df = updated_sot
    
    def match_bank_on_sot_bank_conditions(self, row):
        helper = []
        for col in row.index:
            helper+= ["(updated_sot['{0}'] == row['{0}'])".format(col)]
        return'updated_sot[{}]'.format('&'.join(helper))
    
    def match_bank_on_sot_budget_conditions(self,row):
        date_match = "(abs((updated_sot['budget_date'] - row['Date'])) < pd.to_timedelta('4 Days'))"
        value_match = "(updated_sot['Expense Amount'] == row['Value'])"
        matches = [value_match, date_match]
        return 'updated_sot[{}]'.format('&'.join(matches))
    
    def update_sot(self, row, bank_df, updated_sot):
        # TODO need to think about rare case where updated_sot.groupby(['Description','Date', 'Value']).size().sort_values(ascending=False) 
        matched_bank_cols = eval(self.match_bank_on_sot_bank_conditions(row))
        matched_on_budget = eval(self.match_bank_on_sot_budget_conditions(row))
        one_to_one = (set(matched_bank_cols.index.values) == set(matched_on_budget.index.values))
        
        if matched_bank_cols.shape[0]:  # Existing bank match found, do nothing
            return updated_sot
        elif len(matched_on_budget) == 0:  # No budget match found, make new row
            new_row = pd.DataFrame(data=row, index=row.index).transpose()
            return pd.concat([updated_sot, new_row])        
        elif len(matched_on_budget) == 1 and one_to_one:  # unique budget match found, update entries in SOT
            correct_row = updated_sot
            for key in row.index:
                updated_sot = updated_sot.set_value(matched_bank_cols.index.values[0], updated_sot[key],row[key])
            return updated_sot
        elif len(number_budget_matches) > 1:  # Ambiguous match in budget
            raise Exception('Ambiguous match in budget')
        else:
            raise Exception('Couldnt match budget and bank cols properly')
    

sot = SOT('SOT - 15.12.26 - 16.22.csv')

In [26]:
sot()

Account Account Name        Category       Date  \
0       NaN     'Current             NaN 2015-08-20   
1       NaN     'Current             NaN 2015-08-21   
2       NaN     'Current             NaN 2015-08-21   
3       NaN     'Current             NaN 2015-08-24   
4       NaN     'Current             NaN 2015-08-24   
5       NaN     'Current             NaN 2015-08-26   
6       NaN     'Current             NaN 2015-08-26   
7       NaN     'Current             NaN 2015-08-26   
8       NaN     'Current             NaN 2015-08-27   
9       NaN     'Current             NaN 2015-08-27   
10      NaN     'Current             NaN 2015-08-28   
11      NaN     'Current             NaN 2015-08-28   
12      NaN     'Current             NaN 2015-09-01   
13      NaN     'Current             NaN 2015-09-01   
14      NaN     'Current             NaN 2015-09-01   
15      NaN     'Current             NaN 2015-09-01   
16      NaN     'Current             NaN 2015-09-01   
17      NaN     'Current             NaN 2015-09-01   
18      NaN     'Current             NaN 2015-09-01   
19      NaN     'Current             NaN 2015-09-02   
20      NaN     'Current             NaN 2015-09-02   
21      NaN     'Current             NaN 2015-09-02   
22      NaN     'Current             NaN 2015-09-03   
23      NaN     'Current             NaN 2015-09-04   
24      NaN     'Current             NaN 2015-09-04   
25      NaN     'Current             NaN 2015-09-04   
26      NaN     'Current             NaN 2015-09-04   
27      NaN     'Current             NaN 2015-09-07   
28      NaN     'Current             NaN 2015-09-07   
29      NaN     'Current             NaN 2015-09-07   
..      ...          ...             ...        ...   
245     NaN     'Current             NaN 2015-12-08   
246     NaN     'Current             NaN 2015-12-08   
247     NaN     'Current             NaN 2015-12-09   
248     NaN     'Current             NaN 2015-12-10   
249     NaN     'Current             NaN 2015-12-14   
250     NaN     'Current             NaN 2015-12-14   
251     NaN     'Current             NaN 2015-12-14   
252     NaN     'Current             NaN 2015-12-14   
253     NaN     'Current             NaN 2015-12-14   
254     Rbs     'Current  Food/Groceries 2015-12-14   
255     NaN     'Current             NaN 2015-12-14   
256     Rbs     'Current  Food/Groceries 2015-12-14   
257     Rbs     'Current  Food/Groceries 2015-12-14   
258     Rbs     'Current   Entertainment 2015-12-14   
259     Rbs     'Current   Misc/One-time 2015-12-14   
260     NaN     'Current             NaN 2015-12-14   
261     NaN     'Current             NaN 2015-12-14   
262     Rbs     'Current         Salary  2015-12-15   
263     Rbs     'Current    Departmental 2015-12-15   
264     NaN     'Current             NaN 2015-12-15   
265     NaN     'Current             NaN 2015-12-15   
266     Rbs     'Current        Car/Auto 2015-12-15   
267     NaN     'Current             NaN 2015-12-15   
268     Rbs     'Current  Food/Groceries 2015-12-18   
269     NaN     'Current             NaN 2015-12-18   
270     NaN     'Current             NaN 2015-12-21   
271     NaN     'Current             NaN 2015-12-22   
272     NaN     'Current             NaN 2015-12-23   
273     NaN     'Current             NaN 2015-12-24   
274     NaN     'Current             NaN 2015-12-24   

                                           Description  Expense Amount  Notes  \
0                                    'LBTH COUNCIL TAX             NaN    NaN   
1     '9285 18AUG15 , DELIVEROO.CO.UK , 02033223444 GB             NaN    NaN   
2                                       'NATWEST 20AUG             NaN    NaN   
3    '9285 21AUG15 , AMAZON DIGITAL , DWNLDS , 866-...             NaN    NaN   
4    '9285 23AUG15 , DELIVERANCE , LIMITED , LONDON GB             NaN    NaN   
5    '9285 25AUG15 , UDACITY, INC. , 8778877815 US ...             NaN    NaN   
6          '9285 25AUG15 ,